# Investigating changes in mixed layer depth at Sørvest-F  due to wind farms

__Kjersti Stangeland - Summer 2025 - The Norwegian Meteorological Insitute__

#### The project - Havvind

A scenario with Norwegian wind farms at sea has been developed in a collaboration with NVE. The period 2/2-2022 til 30/6-2022 is simulated with MEPS, WaveWatch III, og Norkyst v3. There are to versions of the simulations:
* A control run based of the current situation with no Norwegian wind farms at sea
* An experiment with simulated wind farms.

In the experiments, the only direct parametrization of the windmills are in MEPS. Therefore, effects on waves and ocean cirulation is indirectly seen through changed surface fluxes.

__Details on the wind farms:__
* Height of turbines is 165 m
* Radius 143 m
* 2.5 km distance between turbines
* 6 (?) different farms along the Norwegian coast

__Sørvest F__
* A wind farm south west of Norway
* 1x1500MW (430 km²) + 2x2100MW (600km² + 600 km²)
* 258 turbines


### Mixed layer depth

The mixed layer of the upper ocean is the first meters of the water column where the potential density is fairly mixed. Below this layer, the density increases almost exponentially. The depth of the mixed layer depends on available energy for mixing, i.e. turbulence ad convection.

In [13]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cmocean.cm as cmo
import numpy as np
import xarray as xr
import cartopy 
from datetime import datetime, timedelta
import pandas as pd
from Rossby_deformation.get_turbine_coords import get_turbine_coords
from netCDF4 import Dataset
from roppy import SGrid
import xroms
from cartopy import config
from glob import glob
from Rossby_deformation.density import dens

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Starting to play around with a test file

In [14]:
def find_indices_of_point(grid, target_lon, target_lat):
    '''
    Finds the [y, x] indices of grid point closest to a given coordinate point (lon, lat).
    The model grid is lon(y, x), lat(y, x).
    '''
    lon = grid.lon_rho
    lat = grid.lat_rho

    y, x = xroms.argsel2d(lon, lat, target_lon, target_lat)

    return y, x

In [15]:
path = '/lustre/storeB/project/nwp/havvind/hav/results/experiment/EXP-03/norkyst_avg_0001.nc'
ds = xroms.open_netcdf(path)
ds, xgrid = xroms.roms_dataset(ds, include_cell_volume=True)

Minimizing the dataset because I just want to test stuff, so need to save some memory.

In [16]:
#ds = ds0.isel(eta_rho=427, xi_rho=183)

In [17]:
pot_dens = dens(ds.salt, ds.temp).squeeze()

__To-do__:

- find out how to calculate MLD - whats the best definition.
- read: https://gmd.copernicus.org/articles/16/3849/2023/gmd-16-3849-2023.pdf


'The MLD may be
computed using a threshold change in density or temperature
(BM04), a threshold in density gradient (Dong et al., 2008), a
maximum density gradient (Large et al., 1997), a maximum in the curvature of the density profile (Lorbacher et al., 2006),
or a minimum of the relative variance (Huang et al., 2018) or
based on energetic principles (Reichl et al., 2022).'

' ...the potential density threshold method has
been recommended by Griffies et al. (2016) to compute the
MLD in OMIP and CMIP models, with a threshold value
of 0.03 kgm−3
.'

what about: 

- read the paper well and see if their method can be used
- make a function which opens file, interpolates from s to z, calculates density, calculates mixed layer depth, writes pot dens and MLD to a netcdf file. run in a loop for all files both ref and exp and save as you did for rossby. 
- start with testing on daily files, and when method works, do it on all.
- then import those files into this notebook and visualize it. 
- ex monthly means and differences
- then you dont have to bother with the limiting study area yet, because you can just do it on the whole.

oh! xroms has xroms.seawater.mld! try that

In [19]:
mld = xroms.mld(pot_dens, xgrid, ds.h, ds.mask_rho)

/home/kjsta7412/.local/lib/python3.9/site-packages/xgcm/grid.py:989: FutureWarning: From version 0.8.0 the Axis computation methods will be removed, in favour of using the Grid computation methods instead. i.e. use `Grid.transform` instead of `Axis.transform`
  warnings.warn(
/home/kjsta7412/.local/lib/python3.9/site-packages/xgcm/grid.py:1026: UserWarning: Input`target_data` has no name, but we need a name for the transformed dimension. The name `TRANSFORMED_DIMENSION` will be used. To avoid this warning, call `.rename` on `target_data` before calling `transform`.
  warnings.warn(


ValueError: cannot add coordinates with new dimensions to a DataArray

In [ ]:
mld

Signature: xroms.mld(sig0, xgrid, h, mask, z=None, thresh=0.03)
Docstring:
Calculate the mixed layer depth [m], return positive and as depth if no value calculated.

Parameters
----------
sig0: DataArray
    Potential density [kg/m^3]
xgrid
    xgcm grid
h: DataArray, ndarray
    Depths [m].
mask: DataArray, ndarray
    mask to match sig0
z: DataArray, ndarray, optional
    The vertical depths associated with sig0. Should be on 'rho'
    grid horizontally and vertically. Use z coords associated with DataArray sig0
    if not input.
thresh: float, optional
    For detection of mixed layer [kg/m^3]

Returns
-------
DataArray of mixed layer depth on rho horizontal grid.
Output is `[T,Y,X]`.

Notes
-----
Mixed layer depth is based on the fixed Potential Density (PD) threshold.

Converted to xroms by K. Thyng Aug 2020 from:

Update history:
v1.0 DL 2020Jun07

References:
ncl mixed_layer_depth function at https://github.com/NCAR/ncl/blob/ed6016bf579f8c8e8f77341503daef3c532f1069/ni/src/lib/nf